# Question 1
Read data from patient notes.

In [ ]:
#%pip install textblob
import pandas as pd
import re
#from textblob import TextBlob
#from spellchecker import SpellChecker

# Read the CSV file
df = pd.read_csv('patient_notes.csv')

# Case Conversion
df['pn_history'] = df['pn_history'].str.lower()

print(df)

### i. Case Conversion and iii.	Correcting Typos and Spelling
Removing Puncutation and Special Characters seems would influence standardizing formatsand handling contractions parts, so it is moved to later.

We tried the TextBlob. But after taking 17 minutes that the first 100 rows have not been handled, we plan to try some other ways.

Then I tried the symspellpy. However, it cannot be installed by conda, only can be dealt with by pip. So I move to SpaCy.

Then I found the pyspellchecker is something I must use with SpaCy and also not able to installed by conda. So I return to symspellpy.

In [ ]:
#pip install symspellpy
from multiprocessing import Pool
import pandas as pd
import numpy as np
from symspellpy import SymSpell, Verbosity
import pkg_resources

# Initialize and load SymSpell
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
bigram_path = pkg_resources.resource_filename("symspellpy", "frequency_bigramdictionary_en_243_342.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

# Function to correct spelling in a sentence
def correct_spelling(text):
    suggestions = sym_spell.lookup_compound(text, max_edit_distance=2)
    return suggestions[0].term if suggestions else text

# Function to apply correct_spelling to a Series
def apply_correct_spelling(series):
    return series.apply(correct_spelling)

# Function to parallelize
def parallelize_dataframe(df, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df_combined = pd.concat(pool.map(apply_correct_spelling, df_split))
    pool.close()
    pool.join()
    return df_combined

# Applying the function in parallel
# df_head = df.head(1000)
if __name__ == 'main':
	df['pn_history'] = parallelize_dataframe(df['pn_history'])
print(df)


### 4th Standardizing Formats 5th Handling Contractions

In [ ]:
# pip install inflect
# pip install contractions
import inflect
import contractions

# Create an engine for inflect
p = inflect.engine()

# Standardizing Formats "17", “17-year-old”, “3-4”, or “17yo”
def convert_numbers_in_string(s):
    # Find all numbers in the string
    numbers_in_words = re.findall(r'\b\d+\b', s)
    numbers_in_parts = re.findall(r'\d+', s)

    # For each number
    for number in numbers_in_words:
        # Convert the number to words
        word = p.number_to_words(number)
        # Replace the number with the word in the string
        s = re.sub(r'\b' + number + r'\b', word, s)

    for number in numbers_in_parts:
        # Convert the number to words
        word = p.number_to_words(number)
        # Replace the number with the word (followed by a space) in the string
        s = re.sub(number, word + " ", s)

    return s

# Handling Contractions: Expanding contractions" 
# "yo", "y.o." "y/o" to "year old",  
# "y" to "year"
# "f" to "female", 
# "m" to "male", 
# "mo" to "month", 
# "yr" to "year"
# "c/o" "c/of" "c/m" to "complains of",
# "cc" to "chief complaint"
# "h/o" to "history of"
# "pt" to "patient"
# "w", "wk" to "week", "hrs" to "hours""
# "x" to times?
def expand_contractions(text):
    # Define a dictionary of contractions and their expanded forms
    custom_contractions = {
        "yo": "year old",
        "y.o.": "year old",
        "y/o": "year old",
        "y": "year",
        "yr": "year",
        "f": "female",
        "m": "male",
        "mo": "month",
        "yr": "year",
        "c/o": "complains of",
        "c/of": "complains of",
        "c/m": "complains of",
        "cc": "chief complaint",
        "h/o": "history of",
        "pt": "patient",
        "w": "week",
        "wk": "week",
        "hrs": "hours",
        "hx": "history",
        "pmh": "past medical history",
        "pmhx": "past medical history",
        "psh": "past surgical history",
        "psurghx": "past surgical history",
        "pshh": "past surgical history",
        "meds": "medications",
        "hosp": "hosipital",
        "fh": "Family history",
        "fhx": "Family history",
        "fmh": "Family history",
        "sh":"social history",
        "soc": "social history",
        "Rx": "prescription",
        "ros":"review of systems",
        "hpi": "history of present illness"
    }

    # First, use the contractions library to expand common English contractions
    text = contractions.fix(text)

    # Then, handle the custom contractions
    # Split the text into words
    words = text.split()
    # For each word in the text
    for i in range(len(words)):
        # If the word is a contraction
        if words[i] in custom_contractions:
            # Replace the contraction with its expanded form in the text
            words[i] = custom_contractions[words[i]]
    # Join the words back into a text string
    return ' '.join(words)



def process_data(df):
    # Fill NaN values with a default value
    df.fillna('Unknown', inplace=True)
    
    # Convert columns to string type
    df['pn_history'] = df['pn_history'].astype(str)

    df['pn_history'] = df['pn_history'].apply(convert_numbers_in_string)

    # Apply the function to the 'pn_history' column
    df['pn_history'] = df['pn_history'].apply(expand_contractions)

    return df

# Apply the function to the DataFrame
df = process_data(df)

# Write the data back to the CSV file
# df.to_csv('data.csv', index=False)
print(df)

### 2nd. Removing Puncutation and Special Characters parts.

In [ ]:
# Removing Punctuation and Special Characters
df['pn_history'] = df['pn_history'].str.replace(r'[^a-zA-Z0-9 ]', ' ', regex=True)

print(df)

### 6th. Stemming
This part would apply the stemming. I choose LancasterStemmer because it seems to be most efficient to find the root and would reduce the number of tokens, which make the last cluster easier.

In [ ]:
import pandas as pd
import re
from nltk.stem import LancasterStemmer

# Create an instance of LancasterStemmer
stemmer = LancasterStemmer()

# Apply LancasterStemmer to the pn_history column
df['pn_history'] = df['pn_history'].apply(lambda x: ' '.join([stemmer.stem(word) for word in re.findall(r'\w+', x)]))

# Print the updated dataframe
print(df)


### 6th. Lemmatizer
Apply lemmatizer.

In [ ]:
import nltk

nltk.download('wordnet')



In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
df['pn_history'] = df['pn_history'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in re.findall(r'\w+', x)]))
print(df)

### 7th. Stop list
Apply a stop word list to filter out unnecessary words.

In [ ]:
from nltk.corpus import stopwords

# Download the stopwords if not already present
nltk.download('stopwords')

# Get the list of stopwords
stop_words = set(stopwords.words('english'))

# Apply stop word filtering to the pn_history column
df['pn_history'] = df['pn_history'].apply(lambda x: ' '.join([word for word in re.findall(r'\w+', x) if word not in stop_words]))

# Print the updated dataframe
print(df)


# Question 2
### a.CountVectorizer
Use CountVectorizer.

In [ ]:
#conda install -c conda-forge scikit-learn
#pip install -U scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Create an instance of CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the "pn_history" column
dtm = vectorizer.fit_transform(df['pn_history'])

# Convert the DTM to a DataFrame
dtm_df = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names_out())

# Display the DTM DataFrame
print(dtm_df)


In [ ]:
total_elements = dtm.shape[0] * dtm.shape[1]
nonzero_elements = dtm.nnz
sparsity = 1 - (nonzero_elements / total_elements)
print('Sparsity of DTM:', sparsity)
dtm_df.info()
print('Memory Usage:', dtm.data.nbytes/ (1024**3), 'GB')


The DTM has 42,146 documents(patient's notes) and 29,081 unique terms. With a sparsity level of approximately 99.72%, it shows that the majority of the matrix elements are zero, meaning each document contains only a small subset of the total terms. Each element represents the frequency of a term in this document and is stored as an int64 type, and the total memory usage of the matrix is around 9.1 GB (If it is stored in csr_matrix, only occupy about 0.026 GB). This highlights the DTM as a highly sparse and memory-intensive structure, which is typical in text analysis involving large text corpora.

### a.TfidfVectorizer
Use TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create an instance of TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the "pn_history" column
dtm_tfidf = vectorizer.fit_transform(df['pn_history'])

# Convert the DTM to a DataFrame
dtm_tfidf_df = pd.DataFrame(dtm_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

# Display the TF-IDF DTM DataFrame
print(dtm_tfidf_df)


In [ ]:
total_elements = dtm_tfidf.shape[0] * dtm_tfidf.shape[1]
nonzero_elements = dtm_tfidf.nnz
sparsity = 1 - (nonzero_elements / total_elements)
print('Sparsity of DTM:', sparsity)
dtm_tfidf_df.info()
print('Memory Usage:', dtm_tfidf.data.nbytes/ (1024**3), 'GB')

The DataFrame has 42,146 rows and 29,081 columns, corresponding to the documents(patient's notes) and terms respectively. The data type of all cells is float64. Each cell represents the TF-IDF score(which is in default setting) of a term in a specific document. The memory usage is substantial at 9.1 GB (If it is stored in csr_matrix, only occupy about 0.026 GB), indicating that while the matrix is sparse, it still occupies a significant amount of memory when converted to a dense format. The sparsity of the DTM is about 99.72%, which means the vast majority of the matrix elements are zeros. This reflects the common case in text data where most terms appear in only a small number of documents. 

### a.n-gram
After studying the n-gram, I think we could try to find gram from bigger to smaller.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk

# Load your CSV file
# Assuming the CSV has a column 'patient_notes' with the text data
patient_notes = df['pn_history'].astype(str)

# Optional: Download NLTK stopwords
# stop_words = set(stopwords.words('english'))

# Initialize CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 4), stop_words=stopwords.words('english'))

# Fit and transform the patient notes
X = vectorizer.fit_transform(patient_notes)

# Convert to DataFrame for better readability (optional)
#dtm = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Now dtm is your document-term matrix with unigrams, bigrams, trigrams, and 4-grams
#print(dtm.head())


In [ ]:
total_elements = X.shape[0] * X.shape[1]
nonzero_elements = X.nnz
sparsity = 1 - (nonzero_elements / total_elements)
print('Sparsity of DTM:', sparsity)
print('DTM Shape:', X.shape)
print('Data Types:', X.dtype)
print('Memory Usage:', X.data.nbytes/ (1024**3), 'GB')  # Convert bytes to GB

This DTM, created using CountVectorizer with an n-gram range of 1 to 4 and English stop words, comprises 42,146 documents(patient's notes) and spans 3,928,802 unique n-grams. Each cell is int64 type and a cell's value indicates how many times the corresponding n-gram appears in that document. It exhibits an extremely high sparsity level of approximately 99.9904%, indicating that the vast majority of the matrix entries are zeros. Despite its vast dimensionality, the actual memory footprint is notably low at around 0.118 GB, thanks to its sparse matrix format, making it efficient for storing and processing such a large-scale text dataset.

# Question 3
### a.Normalization and Outlier Analysis
When we generate TF-IDF, we used 'TfidfVectorizer' from scikit-learn and by defualt, it normalizes all data using L2 norm which means the vector for each document is divided by its Euclidean norm, making the sum of the squares of the vector's elements equal to 1. We don't need to do other normalization again.

In [ ]:
# Outlier analysis
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate Document Lengths
document_lengths = dtm_tfidf_df.sum(axis=1)

# Identify Outlier Documents
Q1 = document_lengths.quantile(0.25)
Q3 = document_lengths.quantile(0.75)
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = document_lengths[(document_lengths < lower_bound) | (document_lengths > upper_bound)]

print(outliers)

#  Visualize Outliers
sns.boxplot(document_lengths)
plt.show()

# Address Outliers
# if you want to remove outliers:
# non_outlier_indices = document_lengths[(document_lengths >= lower_bound) & (document_lengths <= upper_bound)].index
# filtered_dtm_tfidf_df = dtm_tfidf_df.loc[non_outlier_indices]

When analyzing the outlier, we use the sum of TF-IDF scores of each document. Most of documents have a sum of TF-IDF scores that fall within the range represented by the box. The list shows the indices and corresponding TF-IDF sums of the outlier documents. These documents have a TF-IDF sum significantly lower than the rest, falling below the lower bound defined by the whiskers, and hence are flagged as outliers. Possibly because they are shorter in length (number of words) or contain words that are uncommon in the rest of the documents, resulting in lower TF-IDF sums. 

### b.Dimension Reduction
Using dimension reduction techniques such as UMAP, high-dimensional datasets can be reduced into two-dimensional space for visualization and understanding biologically meaningful clusters present in high-dimensional datasets.
#### Parameters:
##### random_state:
As UMAP is a stochastic algorithm, it may produce slightly different results if run multiple times. To reproduce similar results, we use the random_state parameter in UMAP() function.
##### n_neighbors:
This parameter controls how UMAP balances local versus global structure in the data. It does this by constraining the size of the local neighborhood UMAP will look at when attempting to learn the manifold structure of the data.As n_neighbors is increased UMAP manages to see more of the overall structure of the data, gluing more components together, and better coverying the broader structure of the data. By the stage of n_neighbors=15 we have a fairly good overall view of the data showing how they interelate to each other over the whole dataset.
##### min_dist:
The min_dist parameter controls how tightly UMAP is allowed to pack points together. It, quite literally, provides the minimum distance apart that points are allowed to be in the low dimensional representation. This means that low values of min_dist will result in clumpier embeddings.The default value for min_dist is 0.1.
##### n_components:
This parameter allows the user to determine the dimensionality of the reduced dimension space we will be embedding the data into. We can use it for more than just visualisation in 2- or 3-dimensions. If you are interested in (density based) clustering, or other machine learning techniques, it can be beneficial to pick a larger embedding dimension (say 10, or 50) closer to the the dimension of the underlying manifold on which your data lies.
##### metric:
This parameter controls how distance is computed in the ambient space of the input data. The Euclidean metric, specified as metric='euclidean', is one of the most commonly used distance metrics. It calculates the straight-line distance (“as the crow flies”) between two points in space. This metric often makes sense when your features are all measured in compatible units and scales, and when there’s a clear concept of “direction” in your data.

2-dimension

In [ ]:
# pip install umap-learn
import umap.umap_ as umap


# Create an instance of the UMAP model
reducer = umap.UMAP(random_state=42, n_neighbors=15, min_dist=0.1,  n_components=2, metric='euclidean')

# Fit the model to your TF-IDF data and transform the data
umap_embedding = reducer.fit_transform(dtm_tfidf_df)

print(umap_embedding)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(umap_embedding[:, 0], umap_embedding[:, 1])
plt.title('UMAP Results', fontsize=20)
plt.xlabel('UMAP_1')
plt.ylabel('UMAP_2')
plt.show()

In [ ]:
# Statistical Measures
mean_distance = np.mean(np.linalg.norm(umap_embedding, axis=1))
median_distance = np.median(np.linalg.norm(umap_embedding, axis=1))

print(f'Mean distance from origin: {mean_distance}')
print(f'Median distance from origin: {median_distance}')

Mean distance from origin: 10.902891159057617: This is the average Euclidean distance of all data points from the origin in your UMAP reduced space. It’s calculated by summing up the distances of all points from the origin and then dividing by the total number of points. In the context of your assignment, this could indicate that, on average, the physician’s notes (after being processed and reduced in dimensionality) are spread out from the origin in the reduced dimensional space. This might suggest a diversity in the content of the notes.

Median distance from origin: 10.838888168334961: This is the middle value of the Euclidean distances of all points from the origin when these distances are arranged in ascending order. Half of the distances are less than this value and half are greater. The median is a robust measure that is not affected by outliers or extreme values. In the context of your assignment, this tells you that half of your processed notes are within a distance of approximately 10.84 units from the origin in the reduced dimensional space.

These measures give you a sense of how your data (the physician’s notes) is distributed in the reduced dimensional space after applying UMAP. If the mean and median are close to each other (as they are in this case), it suggests that your data might be symmetrically distributed around the origin. 

3-dimension

In [ ]:
# Create an instance of the UMAP model
reducer = umap.UMAP(random_state=42, n_neighbors=15, min_dist=0.1, n_components=3, metric='euclidean')

# Fit the model to your TF-IDF data and transform the data
umap_embedding = reducer.fit_transform(dtm_tfidf_df)

print(umap_embedding)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(umap_embedding[:, 0], umap_embedding[:, 1], umap_embedding[:, 2], s=100)
plt.title('UMAP Results', fontsize=20)
plt.xlabel('UMAP_1')
plt.ylabel('UMAP_2')
plt.show()

In [ ]:
# Statistical Measures
mean_distance = np.mean(np.linalg.norm(umap_embedding, axis=1))
median_distance = np.median(np.linalg.norm(umap_embedding, axis=1))

print(f'Mean distance from origin: {mean_distance}')
print(f'Median distance from origin: {median_distance}')

Mean distance from origin: 11.531579971313477: This is the average Euclidean distance of all data points from the origin in your UMAP reduced space. It’s calculated by summing up the distances of all points from the origin and then dividing by the total number of points. In the context of your assignment, this could indicate that, on average, the physician’s notes (after being processed and reduced in dimensionality) are spread out from the origin in the reduced dimensional space. This might suggest a diversity in the content of the notes.

Median distance from origin: 12.881805419921875: This is the middle value of the Euclidean distances of all points from the origin when these distances are arranged in ascending order. Half of the distances are less than this value and half are greater. The median is a robust measure that is not affected by outliers or extreme values. In the context of your assignment, this tells you that half of your processed notes are within a distance of approximately 10.84 units from the origin in the reduced dimensional space.

These measures give you a sense of how your data (the physician’s notes) is distributed in the reduced dimensional space after applying UMAP. If the mean and median are close to each other (as they are in this case), it suggests that your data might be symmetrically distributed around the origin. 